In [29]:
import pandas as pd
import re, time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def start_driver():
    chrome_options = Options()
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-notifications")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def wanted_crawling_program(start_post, end_post):
    """
    이 함수는 공고를 4회 클릭시 로그인창이 발생하는 사이트의 특성 때문에
    try에서 3개의 공고를 클릭하여 크롤링한 후 
    finally에서 driver를 종료시킨 다음 드라이브를 켜 반복을 이어갑니다.
    """

    url = "https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.recommend_order&years=-1&locations=all"

    wanted_res = pd.DataFrame(
        columns = ['기업명', '주소', '직무', '자격요건', '직급', '이용하는기술스택/우대사항', '해당 페이지 URL']
        )

    for job in range(2, 36):
        driver = start_driver()
        driver.get(url)

        ### 직무 선택 클릭 ###
        search_job = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH, '//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/button/span/span'
                )))
        search_job.click()        
        job_dis = driver.find_element(
            By.XPATH, f'//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/div/div[2]/div[2]/ul[2]/button[{job}]/p'
            ).text
        driver.find_element(
            By.XPATH, f'//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/div/div[2]/div[2]/ul[2]/button[{job}]'
            ).click()
        driver.find_element(
            By.XPATH, '//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/div/div[2]/div[3]/button[2]/span[2]'
            ).click()
        
        time.sleep(1.1)
        ### aws 검색하기 ###
        search = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                    By.XPATH, '//*[@id="__next"]/div[3]/div[1]/section/div[2]/ul/button[2]'
                    )))
        search.click()

        time.sleep(.5)
        for _ in "AWS":      
            time.sleep(0.3)
            driver.find_element(
                By.XPATH, '/html/body/div[3]/div[2]/div/div[1]/div[1]/input'
                ).send_keys(f'{_}')
              
        driver.find_element(
            By.XPATH, '/html/body/div[3]/div[2]/div/footer/div/button[2]/span[2]'
            ).click()

        start_time_scroll = time.time()
        
        # 크롤링 코드블럭
        try:
            for post_job in range(start_post, end_post):
                if time.time() - start_time_scroll > 60:
                    break 
                ### 스크롤하며 공고 찾기 ###
                while True:
                    try:
                        time.sleep(.5)
                        post = driver.find_element(
                            By.XPATH, f'//*[@id="__next"]/div[3]/div[2]/ul/li[{post_job}]/div/a/div[1]'
                            )
                        time.sleep(0.1)
                        try:
                            if post.is_displayed():
                                post.click()
                                break
                        except:
                            driver.execute_script("window.scrollBy(0, 100);")
                            if post.is_displayed():
                                post.click()
                                break
                    except:
                        driver.execute_script("window.scrollBy(0, 100);")
                        if post.is_displayed():
                            post.click()
                            break

                    # 스크롤 시작점=0, ?픽셀씩 내리기
                    driver.execute_script("window.scrollBy(0, 430);")
                    time.sleep(0.6)

                    if time.time() - start_time_scroll > 60:
                        break  
                # while 코드 끝  
                # while 코드에서 클릭한 공고의 data crawling 코드

                # 공고 클릭 후 Data Crawling
                corp = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (
                        By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/a'
                        )
                        )
                        ).text
                time.sleep(0.2)
                #더보기 클릭
                driver.find_element(
                    By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button'
                    ).click()
                time.sleep(0.2)
                tec = driver.find_element(
                    By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[3]/p/span'
                    ).text.replace('\n',', ')
                time.sleep(0.2)
                car = driver.find_element(
                    By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/span[4]'
                    ).text
                time.sleep(0.2)
                post_url = [q.get_attribute('href') for q in driver.find_elements(By.TAG_NAME, 'link')][0]
                time.sleep(0.2)
                main = re.sub(
                    '\s', '', driver.find_element(
                        By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[2]/p/span'
                        ).text
                        ).replace('-', ', ')[1:]
                time.sleep(0.2)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                span_tag = soup.find('span', class_='Typography_Typography__root__xYuMs Typography_Typography__body2__EpxWz Typography_Typography__weightMedium__O0Rdi')
                address = span_tag.get_text()
                
                # Data 인덱스 아래부터 concat
                wanted_temp = pd.DataFrame(
                    [corp, address, job_dis, main, car, tec, post_url],
                    index = ['기업명', '주소', '직무', '자격요건', '직급', '이용하는기술스택/우대사항', '해당 페이지 URL']
                    ).T
                wanted_res = pd.concat(
                    [wanted_res, wanted_temp]
                    )
                
                time.sleep(0.3)
                # 뒤로가기
                driver.back()
                time.sleep(0.3)
        except:
            pass


        finally:
            driver.quit()
            time.sleep(.5)
        
    return wanted_res

In [30]:
res = wanted_crawling_program(1, 5)

In [31]:
res

,기업명,주소,직무,자격요건,직급,이용하는기술스택/우대사항,해당 페이지 URL
0,크라우드웍스,"서울특별시 강남구 역삼동 테헤란로 309 (삼성제일빌딩, 5층)",<selenium.webdriver.remote.webelement.WebEleme...,"경력5년이상서비스및시스템개발경험.•TypeScript및React를이용한Front, ...",경력 5-10년,"• Python 을 이용한 어플리케이션 개발 경험., • Java, Spring B...",https://www.wanted.co.kr/wd/224059
0,매스어답션,서울시 성동구 아차산로9길 21 이레타워 2층,<selenium.webdriver.remote.webelement.WebEleme...,"개발경력5년이상혹은그에준하는역량을갖추신분•HTML,CSS및javascript(ES5...",경력 5-7년,"• 컴퓨터공학 전공 및 관련학과 우대, • 디자인 컴포넌트 시스템 환경에서 개발 경...",https://www.wanted.co.kr/wd/223793
0,에스티씨랩(STCLab),서울특별시 강남구 봉은사로37길 5 (캠프지) 2층,<selenium.webdriver.remote.webelement.WebEleme...,병역특례전문연구요원대상자이신분(현역/보충역)•AWS또는기타클라우드플랫폼에대한실무경험...,경력 5-10년,"• 마이크로서비스 아키텍처 구현에 대한 심층적인 이해와 경험을 갖추신 분, • 멀티...",https://www.wanted.co.kr/wd/224241
0,나니아랩스,LG 사이언스 파크,<selenium.webdriver.remote.webelement.WebEleme...,품질보증관련경력3년이상혹은그에준하는실력을갖추신분•웹및데스크탑앱테스팅기술및경험이있으신...,경력 3-7년,"• 품질 보증 및 테스팅 관련 국가 공인, 국제 자격증(예: ISQTB, CSTS)...",https://www.wanted.co.kr/wd/224078
0,에버엑스,서울 서초구 서리풀길 29-7,<selenium.webdriver.remote.webelement.WebEleme...,백엔드엔지니어와협업경험•Vue또는React를이용하여프로젝트를운영해본경험•Figma사...,경력 3-8년,"• 다양한 JS 프레임워크 경험, • 백엔드(DBA, Infra, BackEnd)업...",https://www.wanted.co.kr/wd/113339
...,...,...,...,...,...,...,...
0,씨메스(CMES),강남구 도산대로 318 SB타워 8층,<selenium.webdriver.remote.webelement.WebEleme...,"경력:3~5년•언어:javascript•framework:threejs,unity,...",경력 3-5년,"• webgl, unreal, opengl 경험자, • 3D 파이프 라인에 대한 이...",https://www.wanted.co.kr/wd/222209
0,파인더스에이아이,"서초구 강남대로 51길 1, 13층(서초동, 511타워)",<selenium.webdriver.remote.webelement.WebEleme...,"Python,PyTorch,TensorFlow•컴퓨터비전(classification...",경력 5-12년,"• 관련 전공 Ph.d, • 뛰어난 커뮤니케이션, writing skills, • ...",https://www.wanted.co.kr/wd/218200
0,이너트립,"인천 부평구 부평대로 293, 903호",<selenium.webdriver.remote.webelement.WebEleme...,5년차이상경력혹은이에준하는실력을갖추신분•Java/javascript언어에능숙하신분•...,경력 5-10년,"- 개발리드 혹은 CTO 포지션의 근무 경험이 있으신 분, - B2B SAAS 인프...",https://www.wanted.co.kr/wd/217776
0,와따,"경기도 성남시 분당구 대왕판교로 670, 유스페이스2 B동 403호",<selenium.webdriver.remote.webelement.WebEleme...,개발팀리드또는개발PM경력5년이상•B2B서비스를시작부터끝까지기획하고출시해보신분•AWS...,경력 5-20년,"• Smart Factory, IOT 플랫폼 관련 경험을 보유하고 열정이 높으신 분...",https://www.wanted.co.kr/wd/218326
